In [1]:
import os
import subprocess
import pandas as pd

In [16]:
# 📂 Rutas
ruta_csv = r"C:/Proyectos/2024/Conejos/mapas/pozos.csv"
ruta_salida = r"C:/Proyectos/2024/Conejos/mapas/interpolados"
os.makedirs(ruta_salida, exist_ok=True)

In [25]:
os.environ["PATH"] += r";C:\OSGeo4W\bin"

subprocess.run(["gdal_grid", "--version"])

CompletedProcess(args=['gdal_grid', '--version'], returncode=0)

In [19]:
# 📌 Ruta del ejecutable de gdal_grid
gdal_grid_exe = r"C:\OSGeo4W\bin\gdal_grid.exe"

In [20]:
# 📌 Configuración
campo_x = "x_nad27"
campo_y = "y_nad27"
años_validos = [2005, 2006, 2007, 2009, 2010, 2011, 2012, 2015, 2016, 2017, 2018] ##

In [21]:
# 📥 Leer CSV con Pandas
df = pd.read_csv(ruta_csv)

In [36]:
for anio in años_validos:
    campo_valor = str(anio)

    datos = df[
        df["x_nad27"].notna() &
        df["y_nad27"].notna() &
        df[campo_valor].notna()
    ][["x_nad27", "y_nad27", campo_valor]]

    if len(datos) < 3:
        print(f"⚠️ Solo {len(datos)} valores válidos para {campo_valor}, se omite.")
        continue

    # Renombrar la columna de valores para que GDAL la acepte
    col_z = f"val_{campo_valor}"
    datos = datos.rename(columns={campo_valor: col_z})

    # Calcular límites
    min_x, max_x = datos["x_nad27"].min(), datos["x_nad27"].max()
    min_y, max_y = datos["y_nad27"].min(), datos["y_nad27"].max()

    # Guardar CSV temporal limpio
    temp_csv = os.path.join(ruta_salida, f"temp_{campo_valor}.csv")
    datos.to_csv(temp_csv, index=False)

    salida_raster = os.path.join(ruta_salida, f"nivel_{campo_valor}.tif")

    # Comando GDAL
    comando = [
        "gdal_grid",
        "-a", "invdist:power=2.0:smoothing=0.0",
        "-txe", str(min_x), str(max_x),
        "-tye", str(max_y), str(min_y),
        "-outsize", "500", "500",
        "-zfield", col_z,
        "-oo", "X_POSSIBLE_NAMES=x_nad27",
        "-oo", "Y_POSSIBLE_NAMES=y_nad27",
        "-of", "GTiff",
        f"CSV:{temp_csv}",
        salida_raster
    ]

    print(f"🔹 Ejecutando: {' '.join(comando)}")
    result = subprocess.run(comando, capture_output=True, text=True)

    if result.returncode != 0:
        print(f"⚠️ Error con {campo_valor}")
        print("STDOUT:", result.stdout.strip())
        print("STDERR:", result.stderr.strip())
    else:
        print(f"✅ Guardado: {salida_raster}")


🔹 Ejecutando: gdal_grid -a invdist:power=2.0:smoothing=0.0 -txe 285776 354918 -tye 3517209 3411260 -outsize 500 500 -zfield val_2005 -oo X_POSSIBLE_NAMES=x_nad27 -oo Y_POSSIBLE_NAMES=y_nad27 -of GTiff CSV:C:/Proyectos/2024/Conejos/mapas/interpolados\temp_2005.csv C:/Proyectos/2024/Conejos/mapas/interpolados\nivel_2005.tif
✅ Guardado: C:/Proyectos/2024/Conejos/mapas/interpolados\nivel_2005.tif
🔹 Ejecutando: gdal_grid -a invdist:power=2.0:smoothing=0.0 -txe 285776 354918 -tye 3517209 3411260 -outsize 500 500 -zfield val_2006 -oo X_POSSIBLE_NAMES=x_nad27 -oo Y_POSSIBLE_NAMES=y_nad27 -of GTiff CSV:C:/Proyectos/2024/Conejos/mapas/interpolados\temp_2006.csv C:/Proyectos/2024/Conejos/mapas/interpolados\nivel_2006.tif
✅ Guardado: C:/Proyectos/2024/Conejos/mapas/interpolados\nivel_2006.tif
🔹 Ejecutando: gdal_grid -a invdist:power=2.0:smoothing=0.0 -txe 285776 354918 -tye 3517209 3411260 -outsize 500 500 -zfield val_2007 -oo X_POSSIBLE_NAMES=x_nad27 -oo Y_POSSIBLE_NAMES=y_nad27 -of GTiff CSV:C:/